In [1]:
from google.colab import files

# Selecciona el ZIP desde tu computador
uploaded = files.upload()

Saving udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip to udea-ai-4-eng-20251-pruebas-saber-pro-colombia (3).zip


In [2]:
import zipfile
import os

# Descomprimir el archivo ZIP
zip_path = "udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip"
extract_path = "datos_saberpro"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Verifica que los archivos estén listos
print("Archivos extraídos:", os.listdir(extract_path))

Archivos extraídos: ['train.csv', 'submission_example.csv', 'test.csv']


In [3]:
import pandas as pd

# Leer archivos ya extraídos
train = pd.read_csv("datos_saberpro/train.csv")
test = pd.read_csv("datos_saberpro/test.csv")
sample_submission = pd.read_csv("datos_saberpro/submission_example.csv")

# Vista previa
train.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [4]:
# ——————————————————————————————
# 3.  Pipeline Saber Pro
# ——————————————————————————————
import pandas as pd, numpy as np, gc, optuna, os, warnings, re
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from category_encoders import CatBoostEncoder
from lightgbm import LGBMClassifier
warnings.filterwarnings("ignore")

# ---------- A.  Carga optimizada ----------
PATH = "datos_saberpro"               # carpeta con los .csv

def read_csv_optimized(path):
    df = pd.read_csv(path, low_memory=False)
    for c in df.columns:
        if df[c].dtype == "float64": df[c] = df[c].astype("float32")
        elif df[c].dtype == "int64": df[c] = df[c].astype("int32")
    return df

train = read_csv_optimized(os.path.join(PATH, "train.csv"))
test  = read_csv_optimized(os.path.join(PATH, "test.csv"))
print(f"✔️ train: {train.shape}   test: {test.shape}")

# ---------- B.  Guardar IDs y flags ----------
test_ids = test["ID"].copy()
train["is_train"] = 1
test ["is_train"] = 0
test["RENDIMIENTO_GLOBAL"] = np.nan     # placeholder

# Quitamos la columna ID para el modelado
train.drop(columns=["ID"], inplace=True)
test.drop(columns=["ID"],  inplace=True)

# ---------- C.  Concatenar ----------
df_all = pd.concat([train, test], axis=0, ignore_index=True)

# ---------- D.  Tipos de columnas ----------
target  = "RENDIMIENTO_GLOBAL"
cat_cols = df_all.select_dtypes("object").columns.tolist()
num_cols = [c for c in df_all.columns if c not in cat_cols + ["is_train", target]]

# ---------- E.  Imputación sencilla ----------
for c in num_cols: df_all[c].fillna(df_all[c].median(), inplace=True)
for c in cat_cols: df_all[c].fillna("missing", inplace=True)

# ---------- F.  Target: texto -> números ----------
label_map = {"bajo": 0, "medio-bajo": 1, "medio-alto": 2, "alto": 3}
df_all[target] = df_all[target].map(label_map)

# ---------- G.  Codificación CatBoost ----------
encoder = CatBoostEncoder(cols=cat_cols, random_state=42, a=0.5)
df_cat_enc = encoder.fit_transform(df_all[cat_cols], df_all[target])

# Combinar con numéricas
df_all_enc = pd.concat([df_cat_enc, df_all[num_cols]], axis=1)
df_all_enc["is_train"] = df_all["is_train"].values
df_all_enc[target]      = df_all[target].values

# ---------- H.  Separar train / test ----------
X      = df_all_enc[df_all_enc["is_train"] == 1].drop(columns=["is_train", target])
y      = df_all_enc[df_all_enc["is_train"] == 1][target]
X_test = df_all_enc[df_all_enc["is_train"] == 0].drop(columns=["is_train", target])

del df_all, df_all_enc, train, test, df_cat_enc; gc.collect()

# ---------- I.  Optuna (10 trials rápidos) ----------
def objective(trial):
    params = {
        "objective": "multiclass",
        "num_class": 4,
        "boosting_type": "gbdt",
        "metric": "multi_logloss",
        "n_estimators": trial.suggest_int("n_estimators", 300, 800),
        "max_depth": trial.suggest_int("max_depth", 4, 12),
        "learning_rate": trial.suggest_float("lr", 0.03, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.7, 1.0),
        "colsample_bytree": trial.suggest_float("colsample", 0.6, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 3.0),
        "random_state": 42,
        "verbosity": -1,
    }
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accs = []
    for tr_idx, val_idx in skf.split(X, y):
        model = LGBMClassifier(**params)
        model.fit(X.iloc[tr_idx], y.iloc[tr_idx])
        preds = model.predict(X.iloc[val_idx])
        accs.append(accuracy_score(y.iloc[val_idx], preds))
    return np.mean(accs)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10, show_progress_bar=True)

best_params = study.best_params
print("🥇 CV‑accuracy mejor:", study.best_value)
best_params.update(dict(objective="multiclass",
                        num_class=4, metric="multi_logloss",
                        random_state=42, verbosity=-1))

# ---------- J.  Entrenamiento final ----------
final_model = LGBMClassifier(**best_params)
final_model.fit(X, y)
test_preds = final_model.predict(X_test)

# ---------- K.  Submission ----------
inv_map = {0: "bajo", 1: "medio-bajo", 2: "medio-alto", 3: "alto"}
submission = pd.DataFrame({
    "ID": test_ids,
    "RENDIMIENTO_GLOBAL": pd.Series(test_preds).map(inv_map)
})
submission.to_csv("submission_catboost_enc.csv", index=False)
print("💾 Generado: submission_catboost_enc.csv — descárgalo desde la barra lateral de Colab")


✔️ train: (692500, 21)   test: (296786, 20)


/tmp/ipython-input-4-3734595062.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  for c in num_cols: df_all[c].fillna(df_all[c].median(), inplace=True)
/tmp/ipython-input-4-3734595062.py:45: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(va

  0%|          | 0/10 [00:00<?, ?it/s]

[I 2025-07-03 15:34:02,174] Trial 0 finished with value: 0.4313444043321299 and parameters: {'n_estimators': 367, 'max_depth': 9, 'lr': 0.11395334775457344, 'subsample': 0.8579545729649598, 'colsample': 0.6992584199127384, 'reg_lambda': 0.6127091513432493}. Best is trial 0 with value: 0.4313444043321299.
[I 2025-07-03 15:47:11,822] Trial 1 finished with value: 0.43142815884476526 and parameters: {'n_estimators': 585, 'max_depth': 10, 'lr': 0.0796650834301414, 'subsample': 0.891995478655582, 'colsample': 0.8507265155332707, 'reg_lambda': 0.19635763120499294}. Best is trial 1 with value: 0.43142815884476526.
[I 2025-07-03 15:57:21,122] Trial 2 finished with value: 0.4294469314079422 and parameters: {'n_estimators': 408, 'max_depth': 5, 'lr': 0.1585466384481937, 'subsample': 0.8649161518030957, 'colsample': 0.849184226962846, 'reg_lambda': 2.252633506041612}. Best is trial 1 with value: 0.43142815884476526.
[I 2025-07-03 16:10:30,466] Trial 3 finished with value: 0.43042888086642594 and p

In [5]:
# Corregir conflicto entre sklearn y numpy
!pip install -U scikit-learn==1.2.2 numpy==1.23.5 --force-reinstall --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 8.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
umap-learn 0.5.8 requires scikit-learn>=1.6, but you have scikit-learn 1.2.2 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.8 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.23.5 which is incompatible.
xarray 2025.3.1 requires numpy>=1.24, but you have nu

In [5]:
# ——————————————————————————————
# 1.  Instalación de dependencias
# ——————————————————————————————
!pip install -U \
    numpy==1.23.5 \
    scikit-learn==1.2.2 \
    lightgbm \
    optuna \
    category_encoders==2.6.0 --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 31.7 MB/s eta 0:00:00
